# <a href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Семинар 6
### **Решающие деревья и лес: повторение**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import seaborn as sns
import scipy.stats
from tqdm import tqdm_notebook
from sklearn import datasets
from sklearn.model_selection import GridSearchCV

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

sns.set(context='poster')
%matplotlib inline

### Задача 1:

Скачайте датасет про классификацию женщин с диабетом. Разделите на трейн и тест.

In [ ]:
data = pd.read_csv("diabetes.csv")

In [ ]:
data

In [ ]:
X = <...>
y = <...>

In [ ]:
X_train, X_test, y_train, y_test = <...>

Обучите обычное решающее дерево глубины 3. Выведите его графическое представление, а также accuracy на тренировочной и тестовой выборке.

In [ ]:
<...>

Посчитайте accuracy

In [ ]:
<...>

Визуализируйте дерево

In [ ]:
<...>

Теперь обучите случайный лес с 100 деревьями.

In [ ]:
n_estimators = 100

<....>

Выведите параметры леса.

In [ ]:
<...>

Какие параметры уже известны и относятся к решающему дереву?

**Ответ:** <...>



Появляются новые важные гиперпараметры: n_estimators (количество деревьев в лесу). Также важным является гиперапараметр max_features &mdash; максимальное количество признаков, которые могут быть перебраны при разбиении вершины дерева. Перед каждым разбиением дерева генерируется выборка из `min(k, max_features)` случайных признаков (`k` — количество признаков в датасете) и только эти признаки рассматриваются как разделяющие в данной вершине.

Выведите качество, сравните с деревом.

In [ ]:
<...>

**Вывод:** <...>

Нарисуйте первые 6 деревьев леса.

In [ ]:
<...>

Как выглядят деревья в решающем дереве? Насколько они отличаются от обученного ранее дерева и друг от друга?

**Вывод:** <...>